The following mini-project intends to use my own Netflix history to determine which limited series (and first seasons) we binge watched since opening my Netflix account and how fast we finished them.

In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict
from pandas.io.json import json_normalize
import requests
import http.client
!conda install -c anaconda beautifulsoup4 --yes
from bs4 import BeautifulSoup
!conda install -c anaconda lxml --yes
import requests
import urllib.request 
#Importing relevant libraries to work with during the project

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    certifi-2020.4.5.1         |           py36_0         159 KB  anaconda
    beautifulsoup4-4.9.0       |           py36_0         165 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.9.0-py36_0      anaconda
    ca-certificates: 2020.1.1-0        --> 2020.1.1-0        anaconda
    certifi:         2020.4.5.1-py36_0 --> 2020.4.5.1-py36_0 anaconda
    openssl:         1.1.1g-

In [2]:
#Importing my downloaded Netflix CSV file into a Pandas Dataframe
data = pd.read_csv('https://raw.githubusercontent.com/escadesupremo/viewing_history/master/NetflixViewingHistory%20(4).csv')
#Splitting Title among ':' for enabling the extraction of Season information
split = data["Title"].str.split(":", n = 3, expand = True)
#Adding watched date information to dataframe
split['Date'] = data['Date']
#Sampling data
split.head(4)

,0,1,2,3,Date
0,The People vs. O.J. Simpson,Season 1,The Run of His Life,None,5/3/20
1,The People vs. O.J. Simpson,Season 1,From the Ashes of Tragedy,None,5/3/20
2,Conan Without Borders,Season 1,Korea,None,5/3/20
3,The Breakfast Club,None,None,None,5/2/20


In order to further cleanse data, season information on both columns 1 and 2 need to be extracted.

In [3]:
#Filtering with Seasons starting with 1
data_1= split[split[1].astype(str).str.contains(' Season 1')]
data_2 = split[split[2].astype(str).str.contains(' Season 1')]
#Renaming columns needed for better readability.
data_2 = data_2.rename(columns={0: 'Title', 2: 'Season'})
data_1 = data_1.rename(columns={0: 'Title', 1: 'Season'})
#Merging title parts back together.
data_2["Title"] = data_2["Title"] + ':' + data_2[1]

result = data_1.append(data_2, sort=False)
#Append one dataframe with the other
result = result[['Title', 'Season', 'Date']]
#Further weed out season starting with 1
result = result[result.Season == ' Season 1']
#Check the bottom 4 results of the dataframe
result.tail(4)

,Title,Season,Date
1812,Time: The Kalief Browder Story,Season 1,9/25/17
1813,Time: The Kalief Browder Story,Season 1,9/25/17
2066,Chef's Table: France,Season 1,3/20/17
2067,Chef's Table: France,Season 1,3/20/17


In order to further refine the data, I will the add the total count (number of episodes watched) as an additional column and remove duplicates from the dataframe.

In [4]:
#Applying date formatting to determine 'number of days watched'
result['Date'] =  pd.to_datetime(result['Date'], format='%m-%d-%y', infer_datetime_format=True)

#Determining the earliest and the latest date for episodes watched for each series
min_series = result.groupby('Title').Date.min()
min_series.rename('MinDate', inplace=True)

max_series = result.groupby('Title').Date.max()
max_series.rename('Max_Depths', inplace=True)

#And turning the results into a dataframe with calculating the number of days passed from the first to last time
minimum_data_frame = pd.concat([min_series, max_series], axis=1)
minimum_data_frame['New'] = minimum_data_frame['Max_Depths'] - minimum_data_frame['MinDate']
minimum_data_frame.head(4)

,MinDate,Max_Depths,New
Title,,,
#blackAF,2020-04-19,2020-04-19,0 days
100 Humans,2020-04-03,2020-04-03,0 days
13 Reasons Why,2017-04-13,2018-05-17,399 days
7 Days Out,2018-12-25,2018-12-29,4 days


In order to further this dataframe, we would need to define the amount of days spent watching each series.

In [5]:
#Finally, merging the dataframes together to ensure clearer understanding.
merged_data = pd.merge(result, minimum_data_frame[['New']], on='Title', how='left')
merged_data = merged_data.sort_values(by=['New'], ascending=False)

#Clearing up formatting to make it look better.
merged_data = merged_data.rename(columns={'count': 'Episodes Watched', 'New': 'Number of Days Watched'})
merged_data = merged_data.drop_duplicates(subset='Title',keep="last")
#Drop date, as it is not needed anymore
merged_data = merged_data.drop(columns=['Date'])
#Sampling results
merged_data.head(5)

,Title,Season,Number of Days Watched
402,Rick and Morty,Season 1,1068 days
656,How I Met Your Mother,Season 1,1016 days
179,The Mind of a Chef,Season 1,911 days
517,Friends,Season 1,843 days
716,Chef's Table: France,Season 1,828 days


Merging the two frames together to achieve further clarity.

In [6]:
#Finally, merging the dataframes together to ensure clearer understanding.
merged_data = pd.merge(result, minimum_data_frame[['New']], on='Title', how='left')
merged_data = merged_data.sort_values(by=['New'], ascending=False)

#Clearing up formatting to make it look better.
merged_data = merged_data.rename(columns={'count': 'Episodes Watched', 'New': 'Number of Days Watched'})
merged_data = merged_data.drop_duplicates(subset='Title',keep="last")
merged_data

,Title,Season,Date,Number of Days Watched
402,Rick and Morty,Season 1,2018-12-07,1068 days
656,How I Met Your Mother,Season 1,2017-04-08,1016 days
179,The Mind of a Chef,Season 1,2019-09-25,911 days
517,Friends,Season 1,2018-01-08,843 days
716,Chef's Table: France,Season 1,2017-03-20,828 days
576,Master of None,Season 1,2017-09-18,712 days
0,The People vs. O.J. Simpson,Season 1,2020-05-03,712 days
491,BoJack Horseman,Season 1,2018-05-05,700 days
261,Ugly Delicious,Season 1,2019-05-29,678 days
560,MINDHUNTER,Season 1,2017-10-26,659 days


In order to gain access to the total number of episodes for each season 1, the Open Movie Database (OMDB API) is used.

First we need to turn our title column into a list, which then can be passed onto the API.

In [9]:
#Putting the Title values from CSV into a list.
requested = result['Title'].values.tolist() 
#Removing all the duplicates from the list.
requested = list(OrderedDict.fromkeys(requested)) 

In [10]:
apikey = '49911e5b'
payload = requested
results = []
Season = '1'
#Above the paramaters for calling API and below the function which calls the API for each element of the list.
for t in payload:
    res = requests.get('http://www.omdbapi.com/?t={}&apikey={}&Season={}'.format(t, apikey, Season))

    if res.status_code == 200:
        results.append(res.json())
    else: 
        print("Request to {} failed".format(t))
        
results[3] #Sampling the results of the API call

Request to #blackAF failed


{'Title': 'Cooked with Cannabis',
 'Season': '1',
 'totalSeasons': '1',
 'Episodes': [{'Title': 'Grilled Backyard BBQ',
   'Released': '2020-04-20',
   'Episode': '1',
   'imdbRating': 'N/A',
   'imdbID': 'tt12176516'},
  {'Title': 'Global Eats',
   'Released': '2020-04-20',
   'Episode': '2',
   'imdbRating': 'N/A',
   'imdbID': 'tt12176518'},
  {'Title': 'I Do Cannabis',
   'Released': '2020-04-20',
   'Episode': '3',
   'imdbRating': 'N/A',
   'imdbID': 'tt12176534'},
  {'Title': 'Futurist Food',
   'Released': '2020-04-20',
   'Episode': '4',
   'imdbRating': '5.3',
   'imdbID': 'tt12176536'},
  {'Title': 'High Holidays',
   'Released': '2020-04-20',
   'Episode': '5',
   'imdbRating': '5.5',
   'imdbID': 'tt12176538'},
  {'Title': 'Comfort Food to High Cuisine',
   'Released': '2020-04-20',
   'Episode': '6',
   'imdbRating': '5.5',
   'imdbID': 'tt12176540'}],
 'Response': 'True'}

In [11]:
dataframe_omdb = json_normalize(results) #Normalizing JSON into a dataframe.
dataframe_omdb.head(5) #Sampling the results.

,Episodes,Error,Response,Season,Title,totalSeasons
0,NaN,Series or season not found!,False,NaN,NaN,NaN
1,NaN,Series or season not found!,False,NaN,NaN,NaN
2,"[{'Title': 'Easy Bake Kevin', 'Released': '201...",NaN,True,1,Real Husbands of Hollywood,5
3,"[{'Title': 'Grilled Backyard BBQ', 'Released':...",NaN,True,1,Cooked with Cannabis,1
4,"[{'Title': 'Getting Defensive', 'Released': '2...",NaN,True,1,Basketball or Nothing,1


The number of episodes watched based on viewing history needs to be determined.

In [16]:
#Creating column with count of episodes watched
result['Episodes Watched'] = result.groupby('Title')['Title'].transform('count')
episodes_watched = result[['Title', 'Episodes Watched']].copy()
episodes_watched = episodes_watched.drop_duplicates(subset='Title',keep="last")

#Merging together with the previously cleansed viewing history data
merged_data = pd.merge(merged_data, episodes_watched, on='Title', how='left')
merged_data = merged_data[['Title', 'Number of Days Watched', 'Episodes Watched']]

#Sampling results
merged_data.tail(4)

,Title,Number of Days Watched,Episodes Watched
183,Marvel's The Punisher,0 days,1
184,Shot in the Dark,0 days,1
185,It's Bruno!,0 days,8
186,The Innocent Man,0 days,6


In [17]:
#Turning episodes into a dataframe series and counting the number of episodes available for each Title.
data_episodecount = dataframe_omdb['Episodes'].apply(pd.Series)
data_episodecount = data_episodecount.count(axis=1)
data_episodecount = pd.DataFrame.from_dict(data_episodecount)

#Merging the two dataframes to ensure more clear understanding
merged_dataAPI = pd.concat([dataframe_omdb, data_episodecount], axis=1)
merged_dataAPI = merged_dataAPI[["Title", "Season", 0]]
merged_dataAPI = merged_dataAPI.rename(columns={0: 'Episodes'})
merged_dataAPI = merged_dataAPI[merged_dataAPI.Episodes != 0]

#Sampling results
merged_dataAPI.head(3)

,Title,Season,Episodes
2,Real Husbands of Hollywood,1,4
3,Cooked with Cannabis,1,6
4,Basketball or Nothing,1,5


Finally merging all frames together to see which TV series have been finished and view the number of days it took to finish it.

In [18]:
merged_data_final = pd.merge(merged_data, merged_dataAPI, on='Title', how='left')
merged_data_final = merged_data_final.dropna()
merged_data_final['Episodes'] = merged_data_final['Episodes'].apply(int)
merged_data_final.tail(8)

,Title,Number of Days Watched,Episodes Watched,Season,Episodes
177,The Indian Detective,0 days,1,1,1
178,Great News,0 days,1,1,7
179,Aussie Gold Hunters,0 days,1,1,2
180,West Coast Customs,0 days,1,1,5
182,The Story of God with Morgan Freeman,0 days,1,1,6
184,Shot in the Dark,0 days,1,1,2
185,It's Bruno!,0 days,8,1,2
186,The Innocent Man,0 days,6,1,1


Lets see which TV series met the original criteria. "Episodes" with 1 total are not TV shows but movies and will be removed from the frame.

In [19]:
merged_data_final2 = merged_data_final[merged_data_final['Episodes Watched'] == merged_data_final['Episodes']]
merged_data_final2 = merged_data_final2[merged_data_final2['Episodes'] != 1]
merged_data_final2

,Title,Number of Days Watched,Episodes Watched,Season,Episodes
0,Rick and Morty,1068 days,11,1,11
4,Chef's Table: France,828 days,4,1,4
7,BoJack Horseman,700 days,12,1,12
24,Narcos,235 days,5,1,5
26,Easy,228 days,8,1,8
28,Million Pound Menu,197 days,6,1,6
53,Cooked with Cannabis,6 days,6,1,6
55,Sex Education,5 days,8,1,8
61,Atypical,3 days,8,1,8
64,Too Hot to Handle,2 days,8,1,8


Upon taking a closer look on this information, it seems that the Open Movie Database is not always correct. There are a number of instances where I have watched 1 more Season 1 episodes than what actually came out. Data is further inaccurate for a number of TV series with more than 1 episode, those are out of scope for this mini project.

In [20]:
merged_data_final3 = merged_data_final[merged_data_final['Episodes Watched'] - 1 == merged_data_final['Episodes']]
merged_data_final3

,Title,Number of Days Watched,Episodes Watched,Season,Episodes
3,Friends,843 days,24,1,23
12,Friends from College,554 days,8,1,7
15,Dirty Money,435 days,6,1,5
42,Bodyguard,14 days,6,1,5
60,On My Block,3 days,10,1,9
68,The Keepers,1 days,7,1,6
74,Champions,1 days,6,1,5
121,Queer Eye: We're in Japan!,0 days,2,1,1
147,Trigger Warning with Killer Mike,0 days,6,1,5


Lets take a look again on the API when we call it for the British series 'Bodyguard'.

In [22]:
apikey = '49911e5b'
payload2 = ['Bodyguard']
results = []
Season = '1'
#Above the paramaters for calling API and below the function which calls the API for each element of the list.
for t in payload2:
    res = requests.get('http://www.omdbapi.com/?t={}&apikey={}&Season={}'.format(t, apikey, Season))

    if res.status_code == 200:
        results.append(res.json())
    else: 
        print("Request to {} failed".format(t))
        
results #Sampling the results of the API call

[{'Title': 'Bodyguard',
  'Season': '1',
  'totalSeasons': '1',
  'Episodes': [{'Title': 'Episode #1.2',
    'Released': '2018-08-27',
    'Episode': '2',
    'imdbRating': '8.7',
    'imdbID': 'tt7658640'},
   {'Title': 'Episode #1.3',
    'Released': '2018-09-02',
    'Episode': '3',
    'imdbRating': '8.4',
    'imdbID': 'tt7658622'},
   {'Title': 'Episode #1.4',
    'Released': '2018-09-09',
    'Episode': '4',
    'imdbRating': '8.2',
    'imdbID': 'tt7658626'},
   {'Title': 'Episode #1.5',
    'Released': '2018-09-16',
    'Episode': '5',
    'imdbRating': '8.3',
    'imdbID': 'tt7658630'},
   {'Title': 'Episode #1.6',
    'Released': '2018-09-23',
    'Episode': '6',
    'imdbRating': '8.5',
    'imdbID': 'tt7517318'}],
  'Response': 'True'}]

As you can see on the results, Season 1 Episode 1 is not retrieved from the Database.

In order to fix this issue, we will scrape the relevant Wikipedia websites for these series in order to determine the total number of Season 1 episodes.

In [23]:
test_link = requests.get('https://next-episode.net/bodyguard/season-1').text #Scraped URL
#Calling BeatifulSoup and looking for specific table
soup  = BeautifulSoup(test_link,'html.parser') 
tables = soup.find_all('div', style='background-color:#f8f8f8;padding:10px;')
#Putting results in a Pandas Frame
df = pd.read_html(str(tables))[0]
#Shaping dataframe to be fit for purpose
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0)).reset_index(drop=True)
df = df.iloc[0:6]
df["Episodes"] = df.shape[0]
#Renaming contents and Putting in Title in order to fit later data analysis
df = df.drop_duplicates(subset='Episodes',keep="last")
df = df.replace({'Episode Name': {'Episode 6': 'Bodyguard'}})
df = df.rename(columns={'Episode Name':'Title'})
df = df[['Title','Episodes']]
df

,Title,Episodes
5,Bodyguard,6


In [24]:
test_link = requests.get('https://en.wikipedia.org/wiki/Trigger_Warning_with_Killer_Mike').text
#Calling BeatifulSoup and looking for specific table
soup  = BeautifulSoup(test_link,'html.parser')
tables3 = soup.find_all('table', class_='wikitable plainrowheaders wikiepisodetable')
#Putting results in a Pandas Frame
df2 = pd.read_html(str(tables3))[0]

#Shaping dataframe to drop not needed data
df2 = df2.rename(columns={'Directed by':'Director'})
df2 = df2[df2.Director == 'Vikram Gandhi']
#Counting number of episodes
df2["Episodes"] = df2.shape[0]
#Further shaping data to be fit for purpose
df2 = df2.drop_duplicates(subset='Episodes',keep="last")
df2 = df2.replace({'Title': {'"Kill Your Master"': 'Trigger Warning with Killer Mike'}})
df2 = df2[["Title","Episodes"]]

df2

,Title,Episodes
8,Trigger Warning with Killer Mike,6


In [25]:
test_link = requests.get('https://en.wikipedia.org/wiki/List_of_Friends_episodes').text
soup  = BeautifulSoup(test_link,'html.parser')
tables3 = soup.find_all('table', class_='wikitable plainrowheaders wikiepisodetable')

df3 = pd.read_html(str(tables3))[0] 

df3["Episodes"] = df3.shape[0] #Episode Count

df3 = df3.drop_duplicates(subset='Episodes',keep="last")
df3 = df3.replace({'Title': {'"The One Where Rachel Finds Out"': 'Friends'}})
df3 = df3[["Title", "Episodes"]]

df3

,Title,Episodes
23,Friends,24


In [26]:
test_link = requests.get('https://en.wikipedia.org/wiki/Friends_from_College').text

soup  = BeautifulSoup(test_link,'html.parser')
tables3 = soup.find_all('table', class_='wikitable plainrowheaders')

df4 = pd.read_html(str(tables3))[0]

df4 = df4.rename(columns={'Season.1':'Title'})
df4 = df4[df4.Title == 1]
df4 = df4.replace({'Title': {1: 'Friends from College'}})
df4 = df4[['Title', 'Episodes']]

df4

,Title,Episodes
0,Friends from College,8


In [27]:
test_link = requests.get('https://en.wikipedia.org/wiki/The_Keepers').text

soup  = BeautifulSoup(test_link,'html.parser')
tables3 = soup.find_all('table', class_='wikitable plainrowheaders wikiepisodetable')

df5 = pd.read_html(str(tables3))[0]

val = ['1','2','3','4','5','6','7'] 
df5 = df5.loc[df5['No.'].isin(val)]
df5["Episodes"] = df5.shape[0] #Episode Count

df5 = df5.drop_duplicates(subset='Episodes',keep="last")
df5 = df5[['Title', 'Episodes']]
df5 = df5.replace({'Title': {'"The Conclusion"': 'The Keepers'}})

df5

,Title,Episodes
12,The Keepers,7


In [28]:
test_link = requests.get('https://en.wikipedia.org/wiki/On_My_Block_(TV_series)').text

soup  = BeautifulSoup(test_link,'html.parser')
tables3 = soup.find_all('table', class_='wikitable plainrowheaders')

df6 = pd.read_html(str(tables3))[0]

df6 = df6.rename(columns={'Season.1':'Title'})
df6 = df6[df6.Title == 1]
df6 = df6.replace({'Title': {1: 'On My Block'}})
df6 = df6[['Title', 'Episodes']]

df6

,Title,Episodes
0,On My Block,10


In [29]:
test_link = requests.get('https://en.wikipedia.org/wiki/Champions_(American_TV_series)').text

soup  = BeautifulSoup(test_link,'html.parser')
tables3 = soup.find_all('table', class_='wikitable plainrowheaders wikiepisodetable')

df7 = pd.read_html(str(tables3))[0]

val = ['1','2','3','4','5','6','7','8','9','10']
df7 = df7.loc[df7['No.'].isin(val)]
df7["Episodes"] = df7.shape[0]

df7 = df7.drop_duplicates(subset='Episodes',keep="last")
df7 = df7[['Title', 'Episodes']]
df7 = df7.replace({'Title': {'"Deal or No Deal"': 'Champions'}})

df7

,Title,Episodes
18,Champions,10


In [30]:
test_link = requests.get('https://en.wikipedia.org/wiki/Dirty_Money_(2018_TV_series)').text

soup  = BeautifulSoup(test_link,'html.parser')
tables3 = soup.find_all('table', class_='wikitable plainrowheaders')

df8 = pd.read_html(str(tables3))[0]

df8 = df8.rename(columns={'Season.1':'Title'})
df8 = df8[df8.Title == 1]
df8 = df8.replace({'Title': {1: 'Dirty Money'}})
df8 = df8[['Title', 'Episodes']]

df8

,Title,Episodes
0,Dirty Money,6


In [31]:
test_link = requests.get('https://en.wikipedia.org/wiki/Queer_Eye_(2018_TV_series)').text

soup  = BeautifulSoup(test_link,'html.parser')
tables3 = soup.find_all('table', class_='wikitable plainrowheaders')

df9 = pd.read_html(str(tables3))[0]

df9 = df9.rename(columns={'Season.1':'Title'})
df9 = df9[df9.Title == '1']
df9 = df9.replace({'Title': {'1': "Queer Eye: We're in Japan!"}})
df9 = df9[['Title', 'Episodes']]

df9

,Title,Episodes
0,Queer Eye: We're in Japan!,8


And lets see together the actual count of those '-1' episodes.

In [32]:
dataframe = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9], ignore_index=True) #Concatenating all dataframes together
dataframe = dataframe.rename(columns={'Episodes':'Actual Episode Count'}) #Differentiation needed for later merging
dataframe #See results

,Title,Actual Episode Count
0,Bodyguard,6
1,Trigger Warning with Killer Mike,6
2,Friends,24
3,Friends from College,8
4,The Keepers,7
5,On My Block,10
6,Champions,10
7,Dirty Money,6
8,Queer Eye: We're in Japan!,8


In [33]:
merged_finalized = pd.merge(merged_data_final3, dataframe, on='Title', how='left') #Merging together the 2 frames

merged_finalized['Episodes'] = merged_finalized['Actual Episode Count'] #Episodes should be actual episode count
merged_finalized = merged_finalized.drop(columns=['Actual Episode Count']) #Drop the duplicated column
merged_finalized = merged_finalized[merged_finalized['Episodes Watched'] == merged_finalized['Episodes']] #Filter for results where we watched the whole episode

merged_finalized #Check results

,Title,Number of Days Watched,Episodes Watched,Season,Episodes
0,Friends,843 days,24,1,24
1,Friends from College,554 days,8,1,8
2,Dirty Money,435 days,6,1,6
3,Bodyguard,14 days,6,1,6
4,On My Block,3 days,10,1,10
5,The Keepers,1 days,7,1,7
8,Trigger Warning with Killer Mike,0 days,6,1,6


In [34]:
frames = [merged_finalized, merged_data_final2] #Defining mergable frames

result = pd.concat(frames) #Concatenating the 2 dataframes
result = result.sort_values(by=['Number of Days Watched'], ascending=True) #Sorting values to put the fastest ones on top

result.head(10) #Check the results

,Title,Number of Days Watched,Episodes Watched,Season,Episodes
176,Huge in France,0 days,8,1,8
8,Trigger Warning with Killer Mike,0 days,6,1,6
72,Kim's Convenience,1 days,13,1,13
69,Surviving R. Kelly,1 days,6,1,6
5,The Keepers,1 days,7,1,7
67,American Vandal,1 days,8,1,8
75,Selling Sunset,1 days,7,1,7
64,Too Hot to Handle,2 days,8,1,8
4,On My Block,3 days,10,1,10
61,Atypical,3 days,8,1,8


I did double check and there are a number of TV series where the OMDB API is inaccurate. Hopefully once they add more episodes and information, running this notebook again will give 100% accurate results. Some real classics like The Innocent Men and It's Bruno should be in here, hopefully once OMDB API is more accurate, it will be indeed.

And finally, lets see our top 3 finishers.

<body>
  <header>
    <h2>Huge in France</h2>
    <img src="http://www.impawards.com/tv/posters/huge_in_france.jpg", alt="Huge in France" width="150" height="75">
    <h2>Trigger Warning with Killer Mike</h2>
    <img src="https://upload.wikimedia.org/wikipedia/en/4/4a/Trigger_Warning_with_Killer_Mike_cover.jpg", alt="Trigger Warning with Killer Mike" width="150" height="75">
    <h2>Kim's Convenience</h2>
    <img src="https://static.wikia.nocookie.net/3f11ef62-865f-4e4f-8c01-02c80891c3d2/scale-to-width/600", alt="Huge in France" width="150" height="75">
  </header>
</body>